# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [4]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sponge_bob_magic.datasets.movielens import MovieLens
from sponge_bob_magic.data_preparator import DataPreparator

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic import metrics

from sponge_bob_magic.models.pop_rec import PopRec
from sponge_bob_magic.models.als_rec import ALSRec
from sponge_bob_magic.models.knn_rec import KNNRec
from sponge_bob_magic.models.lightfm_rec import LightFMRec

from sponge_bob_magic.scenarios.main_scenario import MainScenario
from sponge_bob_magic.session_handler import  get_spark_session
from sponge_bob_magic.constants import DEFAULT_CONTEXT
from pyspark.sql.functions import lit

In [5]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [6]:
spark = get_spark_session()
spark

## Загрузка данных <a name="data-loader"></a>

In [7]:
data = MovieLens("100k")
log = spark.createDataFrame(data.ratings).withColumn(
    "context", lit(DEFAULT_CONTEXT)
)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,...,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,...,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,...,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,...,0,0,1,0,0


## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [8]:
pop_rec = PopRec()
als_rec = ALSRec()
knn_rec = KNNRec()
lightfm_rec = LightFMRec()

In [9]:
log_bydate_splitter = log_splitter.DateSplitter(
    test_start=datetime(2007, 1, 1),
    drop_cold_items=True,
    drop_cold_users=True
)
log_random_splitter = log_splitter.RandomSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)
log_cold_splitter = log_splitter.ColdUsersSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True
)
user_random_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
    
)
user_bydate_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)

### Обучение сценария <a name="fit-scenario"></a>

In [10]:
results = None

In [26]:
scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=lightfm_rec,
    criterion=metrics.HitRate,
    metrics={
        metrics.NDCG: [10,5,3],
        metrics.Precision: [10,5,3],
        metrics.MAP: [10,5,3],
        metrics.Recall: [10,5,3],
        metrics.Surprisal: [10,5,3],
    },
    fallback_rec=pop_rec,
)

In [27]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}
als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}
lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}
knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [29]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

02-Mar-20 12:48:41, root, DEBUG: Деление лога на обучающую и тестовую выборку
02-Mar-20 12:48:55, root, DEBUG: Длина трейна и теста: (84013, 15944)
02-Mar-20 12:48:56, root, DEBUG: Количество пользователей в трейне и тесте: 943, 490
02-Mar-20 12:48:57, root, DEBUG: Количество объектов в трейне и тесте: 1644, 1355
02-Mar-20 12:48:57, root, DEBUG: Инициализация метрик
02-Mar-20 12:48:57, root, DEBUG: Обучение и предсказание дополнительной модели
02-Mar-20 12:48:57, root, DEBUG: Проверка датафреймов
02-Mar-20 12:48:57, root, DEBUG: Предварительная стадия обучения (pre-fit)
02-Mar-20 12:48:57, root, DEBUG: Среднее количество items у каждого user: 90
02-Mar-20 12:49:00, root, DEBUG: Основная стадия обучения (fit)
02-Mar-20 12:49:00, root, DEBUG: Проверка датафреймов
02-Mar-20 12:49:01, root, DEBUG: Количество items после фильтрации: 100
02-Mar-20 12:49:04, root, DEBUG: Пре-фит модели
02-Mar-20 12:49:04, root, DEBUG: -------------
02-Mar-20 12:49:04, root, DEBUG: Оптимизация параметров
02-Ma

In [30]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

number                state     value             datetime_start  \
                                                                     
0  0      TrialState.COMPLETE  0.824490 2020-03-02 12:49:04.904374   
1  1      TrialState.COMPLETE  0.836735 2020-03-02 12:50:00.618402   

           datetime_complete params  \
                               rank   
0 2020-03-02 12:50:00.534919  96      
1 2020-03-02 12:50:55.175552  99      

                                                                 user_attrs  \
                                                                        MAP   
0  {5: 0.4344302721088438, 10: 0.49440848945398286, 3: 0.3863945578231296}    
1  {5: 0.41987641723356034, 10: 0.48229889166550977, 3: 0.3723356009070299}   

                                                                              \
                                                                   Precision   
0  {5: 0.22816326530612244, 10: 0.22000000000000003, 3: 0.23197278911564612}   
1  {5: 0.23265306122448987, 10: 0.22591836734693876, 3: 0.22993197278911556}   

                                                                                \
                                                                        Recall   
0  {5: 0.05015937675062434, 10: 0.09147997078201499, 3: 0.030179258510114822}    
1  {5: 0.051131912575998105, 10: 0.09562767382225167, 3: 0.028924900576843365}   

                                                                             \
                                                                  Surprisal   
0  {5: 0.18488822095928523, 10: 0.19968932311969173, 3: 0.1765698271694075}   
1  {5: 0.18365139229531297, 10: 0.20010869049915095, 3: 0.177733106514085}    

                                                                              \
                                                                        nDCG   
0  {5: 0.23566075600991723, 10: 0.23378266711426007, 3: 0.24064550162129048}   
1  {5: 0.2362501121479612, 10: 0.23723956587707856, 3: 0.23648390796753258}    

  system_attrs  
       _number  
0  0            
1  1

### Получение рекомендаций <a name="predict-scenario"></a>

In [31]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

02-Mar-20 12:52:07, root, DEBUG: Проверка датафреймов
02-Mar-20 12:52:07, root, DEBUG: Предварительная стадия обучения (pre-fit)
02-Mar-20 12:52:08, root, DEBUG: Основная стадия обучения (fit)
02-Mar-20 12:52:08, root, DEBUG: Построение модели LightFM
02-Mar-20 12:52:11, root, DEBUG: Проверка датафреймов
02-Mar-20 12:52:11, root, DEBUG: Выделение дефолтных юзеров
02-Mar-20 12:52:11, root, DEBUG: Выделение дефолтных айтемов


In [32]:
recs.show()

+-------+-------+--------------------+----------+
|user_id|item_id|           relevance|   context|
+-------+-------+--------------------+----------+
|     91|    199|  1.6873581409454346|no_context|
|     91|    300|   1.424494981765747|no_context|
|     91|    510|  1.2464817762374878|no_context|
|     91|    187|  1.0950274467468262|no_context|
|     91|    511|  0.9866685271263123|no_context|
|     91|    661|  0.9003154635429382|no_context|
|     91|    520|  0.8502941727638245|no_context|
|     91|    205|  0.8470395803451538|no_context|
|     91|    526|  0.7202095985412598|no_context|
|     91|    357|  0.5916959047317505|no_context|
|    152|     88|  1.2064160108566284|no_context|
|    152|    393|  0.9332276582717896|no_context|
|    152|     66|  0.7043845057487488|no_context|
|    152|     15|  0.5755499601364136|no_context|
|    152|    237|  0.4729836881160736|no_context|
|    152|    402| 0.45506852865219116|no_context|
|    152|    781| 0.23700232803821564|no_context|
